In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
np.random.seed(42)
pd.set_option('display.width', 1000) 
pd.set_option('display.max_colwidth', None)
import torch

/data/application/anaconda3/envs/pytorch_2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
DATA_DIR = "/xxx/Logic-RL/data/kk/instruct/3ppl/test.parquet"
parquet_data = pd.read_parquet(DATA_DIR)
print(len(parquet_data))
print(parquet_data.iloc[0])
parquet_data = parquet_data[["prompt","solution_text","reward_model","solution"]]
sampled_parquest_data = parquet_data
# sampled_parquest_data = parquet_data.sample(100, random_state=42)


100
quiz                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            A very special island is inhabited only by knights and knaves. Knights always tell the truth, and knaves always lie. You meet 3 inhabitants: Amelia, Penelope, and Evelyn. Amelia said that Evelyn is not a knight. Penelope was heard saying, "Penelope is a knight or Evelyn is a 

In [ ]:
model_base_dir = "/xxx/qwen-25-1_5b"
model_r1_dir = "/xxx/openbookqa_rollout8_8k_entropy0/global_step_620/actor/huggingface"
model_gsm8k_dir = "/xxx/qwen25_1_5b_firstry_10epoch/global_step_580/actor/huggingface"


In [ ]:
device = 'cuda:0'
model_base = AutoModelForCausalLM.from_pretrained(
    model_base_dir,
    torch_dtype=torch.bfloat16,
    device_map= device
)
model_r1 = AutoModelForCausalLM.from_pretrained(
    model_r1_dir,
    torch_dtype=torch.bfloat16,
    device_map= device
)
model_gsm8k = AutoModelForCausalLM.from_pretrained(
    model_gsm8k_dir,
    torch_dtype=torch.bfloat16,
    device_map= device
)
tokenizer = AutoTokenizer.from_pretrained(model_base_dir)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [ ]:

import re
def extra_answer(text):
    pattern = r'<answer>(.*?)</answer>'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        result = match.group(1)
    else:
        result = text
    print(result)
    pattern = r'([A-Z]\w+)\s+is\s+(a\s+)?(\w+)'
    matches = re.findall(pattern, result)
    result_dict = {name: ' '.join(match[1:]).strip() for name, *match in matches}
    return result_dict
    

def get_score(text, soluntion, name=""):
    result_dict = extra_answer(text)
    answer_dict  = extra_answer(soluntion)
    print(f"[get_score] {name}: ", result_dict, answer_dict)
    score = 0
    for k, v in answer_dict.items():
        if k in result_dict:
            if result_dict[k] == v:
                score += 1
    return score

all_data = []
temper = 0.6
for i in range(len(sampled_parquest_data)):
    prompt = sampled_parquest_data.iloc[i]["prompt"]
    solution_text = sampled_parquest_data.iloc[i]["solution_text"]
    ground_truth = sampled_parquest_data.iloc[i]["reward_model"]["ground_truth"]
    solution = sampled_parquest_data.iloc[i]["solution"]
    text = tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model_base.device)

    generated_ids_base = model_base.generate(
        **model_inputs,
        max_new_tokens=768,
        temperature=temper,
    )
    generated_ids_base = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids_base)
    ]

    response_base = tokenizer.batch_decode(generated_ids_base, skip_special_tokens=True)[0]
    base_score = get_score(response_base, solution_text,name="base")  
    # print("response_base:",response_base, len(generated_ids_base[0]))
    # print("response_base:", response_base)
    #=================================
    generated_ids_r1 = model_r1.generate(
        **model_inputs,
        max_new_tokens=768,
        temperature=temper,
        repetition_penalty=1.1
    )
    generated_ids_r1 = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids_r1)
    ]
    response_r1 = tokenizer.batch_decode(generated_ids_r1, skip_special_tokens=True)[0]
    r1_score = get_score(response_r1, solution_text,name="r1") 
    # print("response_r1:",response_r1,len(generated_ids_r1[0]))
    #=================================
    generated_ids_gsm8k = model_gsm8k.generate(
        **model_inputs,
        max_new_tokens=768,
        temperature=temper,
        repetition_penalty=1.1
    )
    generated_ids_gsm8k = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids_gsm8k)
    ]
    response_gsm8k = tokenizer.batch_decode(generated_ids_gsm8k, skip_special_tokens=True)[0]
    gsm8k_score = get_score(response_gsm8k, solution_text, name="gsm8k") 
    #print("response_gsm8k:",response_gsm8k,len(generated_ids_gsm8k[0]))
    all_data.append([prompt,solution_text, response_base, response_r1, response_gsm8k,
                     base_score, r1_score, gsm8k_score,
                     len(generated_ids_base[0]),len(generated_ids_r1[0]),len(generated_ids_gsm8k[0])])
    print(i, len(generated_ids_base[0]), len(generated_ids_r1[0]), len(generated_ids_gsm8k[0]))
    print("score: ", base_score, r1_score, gsm8k_score)
    print('='*10)
    # print("="*20)
    # if i > 4:break 

(1) Amelia is a knight
Amelia is a knight, Penelope is a knight, and Evelyn is a knave.
[get_score] base:  {'Amelia': 'knight'} {'Amelia': 'knight', 'Penelope': 'knight', 'Evelyn': 'knave'}
(1) Amelia is a knave
(2) Penelope is a knave
(3) Evelyn is a knight
Amelia is a knight, Penelope is a knight, and Evelyn is a knave.
[get_score] r1:  {'Amelia': 'knave', 'Penelope': 'knave', 'Evelyn': 'knight'} {'Amelia': 'knight', 'Penelope': 'knight', 'Evelyn': 'knave'}
 (1) Amelia is a knight
(2) Penelope is a knave
(3) Evelyn is a knave 
Amelia is a knight, Penelope is a knight, and Evelyn is a knave.
[get_score] gsm8k:  {'Amelia': 'knight', 'Penelope': 'knave', 'Evelyn': 'knave'} {'Amelia': 'knight', 'Penelope': 'knight', 'Evelyn': 'knave'}
0 517 549 516
score:  1 0 2
Let's analyze the statements made by each inhabitant step by step:

1. **Michael**: 
   - Statement: "If Victoria is a knave, then Evelyn is a knight."
   - This statement implies two possible scenarios:
     - Scenario 1: Victor

In [7]:
print(all_data[0])

[array([{'content': '<|im_start|>system\nYou are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and<answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>.  Now the user asks you to solve a logical reasoning problem. After thinking, when you finally reach a conclusion, clearly state the identity of each character within <answer> </answer> tags. i.e., <answer> (1) Zoey is a knight\n(2) ... </answer>.\n<|im_end|>\n<|im_start|>user\nA very special island is inhabited only by knights and knaves. Knights always tell the truth, and knaves always lie. You meet 3 inhabitants: Amelia, Penelope, and Evelyn. Amelia said that Evelyn is not a knight. Penelope was heard saying, "Penelope is a knight or Evelyn is a knave". Evelyn was heard saying, "Penelope is a knight if and only if Penelo

In [9]:
save_data = pd.DataFrame(all_data, columns=["question", "answer", "response_base", "response_rl_choice", "response_rl_gsm8k",
                                             "score_base", "score_rl_choice", "score_rl_gsm8k","len_base", "len_rl_choice", "len_rl_gsm8k"])

In [16]:
print(np.mean(save_data["score_base"]), np.mean(save_data["score_rl_choice"]), np.mean(save_data["score_rl_gsm8k"]))

1.26 1.48 1.61


In [15]:
print(np.mean(save_data["len_base"]), np.mean(save_data["len_rl_choice"]), np.mean(save_data["len_rl_gsm8k"]))

478.93 469.66 453.25


In [ ]:
idx = 1
print(save_data['question'].iloc[idx])
print('='*10)
print(save_data['response_base'].iloc[idx])
print('='*10)
print(save_data["response_rl_choice"].iloc[idx])

What part of a plant protects the plant cell, maintains its shape, and prevents excessive uptake of water?
To determine what part of a plant protects the plant cells, maintains their shape, and prevents excessive uptake of water, let's break down these functions one by one:

1. **Protecting Plant Cells:**
   - The primary protective layer in plants is the cuticle on the outer surface of leaves and stems. This cuticle acts as a barrier against pathogens, insects, and extreme environmental conditions like high humidity or low temperatures.

2. **Maintaining Shape:**
   - In most plants, especially woody ones, the main structural support comes from secondary xylem (wood). Secondary xylem provides strength and rigidity to the stem, helping it maintain its upright shape even under heavy loads.

3. **Preventing Excessive Uptake of Water:**
   - To prevent excessive water absorption, some plants have specialized structures called guard cells in their stomata. Guard cells control the opening a

In [17]:
save_data.to_csv("kk_base_rl_response_ent0_penalty1_1_temp06.csv", index=False)